In [7]:
# Standard libraries
import datetime
import math
import pathlib
import sqlite3
import sys

# Installed libraries
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import spiceypy

# Append to root directory of this repository
sys.path.append("../")

# Auxiliary module that contains the apparent magnitude
from auxiliary import photometry

In [2]:
# Accessing the NEO database
database_dir = pathlib.Path("../databases/neos/")
database_file = pathlib.Path("neodys.db")
database_filepath = database_dir / database_file

# Establish a connection to the database and set a cursor
neodys_db_con = sqlite3.connect(database_filepath)
neodys_db_cur = neodys_db_con.cursor()

In [3]:
# Get all information from the DB. Since the DB is rather small, this won't cause any issues!
neo_df = pd.read_sql("SELECT * FROM main", neodys_db_con)

# Close the database.
neodys_db_con.close()

In [29]:
neo_df

,Name,Epoch_MJD,SemMajAxis_AU,Ecc_,Incl_deg,LongAscNode_deg,ArgP_deg,MeanAnom_deg,AbsMag_,SlopeParamG_,Aphel_AU,Perihel_AU,NEOClass,Perihel_km,Incl_rad,LongAscNode_rad,ArgP_rad,MeanAnom_rad,Epoch_JD,Epoch_et
0,433,59600.0,1.458273,0.222727,10.828461,304.296346,178.897177,246.904119,10.87,0.46,1.783070,1.133476,Amor,1.695656e+08,0.188992,5.310973,3.122345,4.309290,2459600.5,6.959953e+08
1,719,59600.0,2.637505,0.546959,11.575272,183.855259,156.227573,278.197185,15.53,0.15,4.080112,1.194899,Amor,1.787543e+08,0.202027,3.208880,2.726686,4.855457,2459600.5,6.959953e+08
2,887,59600.0,2.473158,0.570486,9.394141,110.428769,350.493276,86.607207,13.33,-0.12,3.884060,1.062256,Amor,1.589113e+08,0.163959,1.927346,6.117262,1.511581,2459600.5,6.959953e+08
3,1036,59600.0,2.665849,0.533122,26.677908,215.517191,132.428118,140.654843,9.41,0.30,4.087071,1.244627,Amor,1.861936e+08,0.465617,3.761485,2.311307,2.454890,2459600.5,6.959953e+08
4,1221,59600.0,1.918731,0.435843,11.883264,171.320251,26.643922,261.044505,17.40,0.15,2.754996,1.082467,Amor,1.619347e+08,0.207402,2.990102,0.465024,4.556086,2459600.5,6.959953e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29044,2022KY2,59600.0,1.439076,0.290852,9.562666,220.831102,344.748373,308.992894,24.82,0.15,1.857634,1.020519,Amor,1.526674e+08,0.166900,3.854230,6.016994,5.392943,2459600.5,6.959953e+08
29045,2022KZ,59600.0,1.404505,0.389038,10.755270,64.554335,243.245936,257.305079,26.44,0.15,1.950911,0.858099,Apollo,1.283698e+08,0.187715,1.126686,4.245442,4.490821,2459600.5,6.959953e+08
29046,2022KZ1,59600.0,1.643745,0.562916,31.322592,63.673736,262.310279,276.670518,26.14,0.15,2.569035,0.718455,Apollo,1.074793e+08,0.546682,1.111316,4.578178,4.828811,2459600.5,6.959953e+08
29047,2022KZ2,59600.0,1.342108,0.412424,13.556790,237.807367,283.141272,319.777066,26.25,0.15,1.895626,0.788590,Apollo,1.179713e+08,0.236611,4.150522,4.941747,5.581163,2459600.5,6.959953e+08


In [19]:
spiceypy.furnsh("../kernels/spk/de432s.bsp")
spiceypy.furnsh("../kernels/lsk/naif0012.tls")
spiceypy.furnsh("../kernels/pck/gm_de431.tpc")

# Get the G*M value of the Sun
_, gm_sun_pre = spiceypy.bodvcd(bodyid=10, item='GM', maxn=1)
gm_sun = gm_sun_pre[0]

In [13]:
curr_time_et = spiceypy.utc2et(datetime.datetime.now().strftime("%Y-%m-%d"))

In [48]:
sun2earth_position_vec = spiceypy.spkgps(targ=399,
                                     et=curr_time_et,
                                     ref="ECLIPJ2000",
                                     obs=10)[0]
sun2earth_position_vec

array([ 1.50093320e+08,  1.64316490e+06, -1.29773492e+03])

In [33]:
neo_df.loc[:, "Perihel_km"] = neo_df["Perihel_AU"].apply(lambda x: spiceypy.convrt(x, "AU", "km"))
neo_df.loc[:, "Incl_rad"] = neo_df["Incl_deg"].apply(lambda x: math.radians(x))
neo_df.loc[:, "LongAscNode_rad"] = neo_df["LongAscNode_deg"].apply(lambda x: math.radians(x))
neo_df.loc[:, "ArgP_rad"] = neo_df["ArgP_deg"].apply(lambda x: math.radians(x))
neo_df.loc[:, "MeanAnom_rad"] = neo_df["MeanAnom_deg"].apply(lambda x: math.radians(x))
neo_df.loc[:, "Epoch_JD"] = neo_df["Epoch_MJD"].apply(lambda x: x + 2400000.5)
neo_df.loc[:, "Epoch_et"] = neo_df["Epoch_JD"].apply(lambda x: spiceypy.utc2et(str(x) + " JD"))

In [43]:
neo_df.loc[:, "sun2neo_position_vec"] = \
    neo_df.apply(lambda x: spiceypy.conics(elts=[x["Perihel_km"],
                                                 x["Ecc_"],
                                                 x["Incl_rad"],
                                                 x["LongAscNode_rad"],
                                                 x["ArgP_rad"],
                                                 x["MeanAnom_rad"],
                                                 x["Epoch_et"],
                                                 gm_sun],
                                         et=curr_time_et)[:3],
                 axis=1)

In [46]:
neo_df.iloc[0]["sun2neo_position_vec"]

array([-1.66166744e+08,  5.62640353e+07, -2.01934353e+07])

In [50]:
neo_df.loc[:, "neo2earth_position_vec"] = \
    neo_df["sun2neo_position_vec"].apply(lambda x: sun2earth_position_vec - x)
neo_df.loc[:, "neo2sun_position_vec"] = \
    neo_df["sun2neo_position_vec"].apply(lambda x: -1.0 * x)

In [54]:
neo_df.loc[:, "neo2earth_position_vec_AU"] = \
    neo_df["neo2earth_position_vec"].apply(lambda x: [spiceypy.convrt(k, "km", "AU") for k in x])
neo_df.loc[:, "neo2sun_position_vec_AU"] = \
    neo_df["neo2sun_position_vec"].apply(lambda x: [spiceypy.convrt(k, "km", "AU") for k in x])

In [55]:
neo_df.loc[:, "app_mag"] = \
    neo_df.apply(lambda x: photometry.hg_app_mag(abs_mag=x["AbsMag_"],
                                                 vec_obj2obs=x["neo2earth_position_vec_AU"],
                                                 vec_obj2ill=x["neo2sun_position_vec_AU"],
                                                 slope_g=x["SlopeParamG_"]), axis=1)

In [57]:
neo_df[["Name", "app_mag"]]

,Name,app_mag
0,433,13.290833
1,719,19.527435
2,887,20.360448
3,1036,16.235524
4,1221,19.483616
...,...,...
29044,2022KY2,27.134045
29045,2022KZ,27.919159
29046,2022KZ1,28.624954
29047,2022KZ2,29.567487


In [64]:
neo_df.loc[neo_df["app_mag"] == neo_df.app_mag.min()]

,Name,Epoch_MJD,SemMajAxis_AU,Ecc_,Incl_deg,LongAscNode_deg,ArgP_deg,MeanAnom_deg,AbsMag_,SlopeParamG_,...,ArgP_rad,MeanAnom_rad,Epoch_JD,Epoch_et,sun2neo_position_vec,neo2earth_position_vec,neo2sun_position_vec,app_mag,neo2earth_position_vec_AU,neo2sun_position_vec_AU
7444,2010LF64,59600.0,1.343113,0.166077,17.389129,19.028392,239.901061,151.96498,-9.99,0.15,...,4.187063,2.652289,2459600.5,6.959953e+08,"[-181445907.37038097, -32345143.140564624, 895...","[331539227.3788605, 33988308.04103009, -895186...","[181445907.37038097, 32345143.140564624, -8950...",-7.404241,"[2.2162028511422083, 0.22719780636984552, -0.0...","[1.2128909764961446, 0.21621392743009338, -0.0..."
